In [1]:
import gymnasium as gym
import tensorflow as tf
import numpy as np
import random

# Define the CNN model
def create_model():
    #inputs = tf.keras.Input(shape=(4,))
    inputs = tf.keras.Input(shape=(8,)) #changed
    x = tf.keras.layers.Dense(128, activation="relu")(inputs)
    x = tf.keras.layers.Dense(128, activation="relu")(x)
    #outputs = tf.keras.layers.Dense(2, activation="relu")(x)
    outputs = tf.keras.layers.Dense(4, activation="relu")(x) #changed
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="LunarLander")
    return model

optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)



2023-06-13 19:14:37.554967: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 19:14:39.878587: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [2]:
class Environment:
    def __init__(self, NUM_ENVS, EPSILON = 0.2):
        self.num_envs = NUM_ENVS
        #self.envs = envs = gym.vector.make('CartPole-v1', num_envs=NUM_ENVS)#, render_mode='human') #changed
        self.envs = envs = gym.vector.make('LunarLander-v2', num_envs=NUM_ENVS)
        self.current_state, _ = self.envs.reset()
        self.epsilon = EPSILON

    def sample(self, model):
        q_values = model(self.current_state) #get q values for current state
        action = np.argmax(q_values, axis=1) #get action that maximizes q-value #potential error?
        action = [random.choice(range(4)) if random.random() < self.epsilon else a for a in action] #choose epsilon greedy #changed
        new_observation, reward, terminated, truncated, info = self.envs.step(action)
        old_observation = self.current_state

        self.current_state = new_observation
        return (old_observation, action, reward, new_observation, terminated)


In [3]:
class Buffer:
    def __init__(self, max_size):
        self.buffer = []
        self.max_size = max_size

    def add_to_buffer(self, samples):
        #unpack the different environments
        old_obs_ = samples[0]
        actions_ = samples[1]
        rewards_ = samples[2]
        new_obs_ = samples[3]
        terminateds_ = samples[4]
        for o, a, r, no, t in zip(old_obs_, actions_, rewards_, new_obs_, terminateds_):
            self.buffer.append((o, a, r, no, t))
        if len(self.buffer) > self.max_size:
            self.buffer = self.buffer[-self.max_size:]

    def sample_minibatch(self, batch_size):
        r = random.sample(self.buffer, min(len(self.buffer), batch_size))
        return r
        

In [4]:
TAU = 0.01
N = 5
K = 3
MINI_BATCH_SIZE = 64
CONVERGED = False
#MAX_STEPS = 2000#0
MAX_STEPS = 2000
GAMMA = 0.98


In [5]:

envs = Environment(7)

# Create the model
Q_theta = create_model()
Q_target = tf.keras.models.clone_model(Q_theta)

kabuff = Buffer(100000)

STEPS = 0
while not CONVERGED and STEPS < MAX_STEPS:
    STEPS += 1
    
    Q_target.set_weights(Q_theta.get_weights())
    
    for n in range(N):
        kabuff.add_to_buffer(envs.sample(Q_theta))
        for k in range(K):
            #sample s,a,r,s' minibatch from buffer
            minibatch = kabuff.sample_minibatch(MINI_BATCH_SIZE)

            new_states = [sample[3] for sample in minibatch]
            rewards = [sample[2] for sample in minibatch]
            actions = [sample[1] for sample in minibatch]
            old_states = [sample[0] for sample in minibatch]
            terminateds = [sample[4] for sample in minibatch]

            new_states = np.array(new_states)
            rewards = np.array(rewards)
            actions = np.array(actions)
            old_states = np.array(old_states)
            terminateds = np.array(terminateds)
            #everything seems fine

            Q_target_values = Q_target(new_states)
            #print("new states:", new_states)
            #print("Q_target_values", Q_target_values)
            #print("----------------------------------------------------------------------")
            #weirdly learns many zeros
            max_Q_target_values = np.array([max(action_values) for action_values in Q_target_values])
            target_q_values = rewards + GAMMA * max_Q_target_values #this calculation works

            for i, tqv in enumerate(target_q_values):
                if terminateds[i]:
                    target_q_values[i] = rewards[i]

            with tf.GradientTape() as tape:
                predictions = Q_theta(old_states)
                #print(predictions)
                selected_q_values = tf.gather(predictions, actions, batch_dims=1)
                loss_value = tf.reduce_mean(tf.square(selected_q_values-target_q_values))
            gradients = tape.gradient(loss_value, Q_theta.trainable_variables)
            optimizer.apply_gradients(zip(gradients, Q_theta.trainable_variables))
    if STEPS % 100 == 0:
        print("last loss " + str(loss_value) + " in step " + str(STEPS))

last loss tf.Tensor(89.74527, shape=(), dtype=float32) in step 100
last loss tf.Tensor(42.746784, shape=(), dtype=float32) in step 200
last loss tf.Tensor(73.68666, shape=(), dtype=float32) in step 300
last loss tf.Tensor(35.2081, shape=(), dtype=float32) in step 400
last loss tf.Tensor(11.929905, shape=(), dtype=float32) in step 500
last loss tf.Tensor(14.597083, shape=(), dtype=float32) in step 600
last loss tf.Tensor(45.5531, shape=(), dtype=float32) in step 700
last loss tf.Tensor(203.23941, shape=(), dtype=float32) in step 800
last loss tf.Tensor(14.746721, shape=(), dtype=float32) in step 900
last loss tf.Tensor(0.49103487, shape=(), dtype=float32) in step 1000
last loss tf.Tensor(28.29597, shape=(), dtype=float32) in step 1100
last loss tf.Tensor(17.140133, shape=(), dtype=float32) in step 1200
last loss tf.Tensor(6.564639, shape=(), dtype=float32) in step 1300
last loss tf.Tensor(25.736004, shape=(), dtype=float32) in step 1400
last loss tf.Tensor(279.30627, shape=(), dtype=flo

In [6]:
Q_theta.save('some_basic_model')

INFO:tensorflow:Assets written to: some_basic_model/assets


In [ ]:
Q_theta = keras.models.load_model("some_basic_model")

In [7]:
test_env = gym.make('LunarLander-v2', render_mode='human')
#test_env = gym.make('CartPole-v1', render_mode='human')
obs, inf = test_env.reset()


In [10]:
for i in range(1000):
    qs = Q_theta(tf.expand_dims(obs, 0))
    act = np.argmax(qs)
    obs, _, terminated, _, _ = test_env.step(act)
    if(terminated):
        obs, _ = test_env.reset()

: 